In [1]:
import pandas as pd
from tech_core.feature_pipeline import FeaturesPipeline

path_to_data = '../data/'

In [6]:
# from tqdm import tqdm
cmf_dim = asset_dim = num_assets = 488
# for common_feats, asset_feats, fut_ret in tqdm(reader):
#     cmf_dim = common_feats.shape[1]
#     asset_dim = asset_feats.shape[2]
#     num_assets = asset_feats.shape[1]
#     break

In [7]:
from tech_core.losses import StreamingSharpeLoss
from tech_core.nn_builder import SimplePortfolioAllocator, DeepPortfolioAllocator
import torch

split_dates = [
    pd.Timestamp("2025-01-01")
]
split_names = ['train', 'test']

model = SimplePortfolioAllocator(
    cmf_dim=cmf_dim,
    asset_dim=asset_dim,
    num_assets=num_assets,
    hidden_cmf=64,
    hidden_asset=32,
    head_hidden=16
)

pipeline = FeaturesPipeline(path_to_data, padding=35, chunk_size=1000,
                        split_dates=split_dates,
                        split_names=split_names)

# Big NN
# Sharpe Ratio (MSE-guided) for the epoch: 0.0106
# Total MSE loss for the epoch: 0.0000007199
# Pnl : 0.2

import torch
from tqdm import tqdm
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
num_epochs = 100 # Количество эпох обучения
fees = 1.5e-4
for epoch in range(num_epochs):
    loss_fn = StreamingSharpeLoss(fee=fees)
    loss_fn.reset()
    pipeline.reset()
    for common_feats, asset_feats, fut_ret in tqdm(pipeline.iterate('train'), desc=f"Epoch {epoch+1}"):

        # Перевод в тензоры
        common_feats = torch.tensor(common_feats.values, dtype=torch.float32).to(device)       # (T, d_common)
        asset_feats  = torch.tensor(asset_feats,         dtype=torch.float32).to(device)       # (T, n_assets, d_asset)
        fut_ret      = torch.tensor(fut_ret.values,      dtype=torch.float32).to(device)       # (T, n_assets)

        if len(common_feats) != 1000:
            continue

        optimizer.zero_grad()

        # Вычисляем веса
        weights = model(common_feats, asset_feats)  # (T, n_assets)
        # Обновляем метрику
        loss_value = loss_fn.forward(
            weights=weights,
            returns=fut_ret
        )
        loss_value.backward()

        optimizer.step()

    loss_fn.plot_whole_epoch_loss()  # Выводим Sharpe Ratio за эпоху

AttributeError: 'FastCSVChunkReader' object has no attribute 'end_idx'